# Savvy Zealy Airdrop: Round 1

In [1]:
import json
import pandas as pd
import numpy as np

I scraped a snapshot of Zealy using their API on November 13, 2023.

First load and merge the snapshot. We conclude with the total count of users in the snapshot.

In [2]:
with open('data/p0.json', 'r') as f:
    df0 = pd.DataFrame.from_dict(json.load(f)['leaderboard'], orient='columns')

with open('data/p1.json', 'r') as f:
    df1 = pd.DataFrame.from_dict(json.load(f)['leaderboard'], orient='columns')

df = pd.concat([df0, df1])
len(df)

1820

Count the number of Zealy users with blank addresses.

In [3]:
df['address'].replace('', np.nan, inplace=True)
df['address'].isna().sum()

14

In [4]:
df['connectedWallet'].replace('', np.nan, inplace=True)
df['connectedWallet'].isna().sum()

223

Try to recover addresses by examining connectedWallet, too.

In [5]:
df['addr'] = df['address'].fillna(df['connectedWallet'])
df['addr'].isna().sum()

14

Unfortunately, we still have the same number of missing addresses - so none were recovered with this method.

Since we cannot airdrop to accounts that never provided addresses, we will drop them to keep the calculations in order.

In [6]:
df = df.dropna(subset=['address'])
len(df)

1806

Drop any columns that are not necessary for the airdrop.

In [7]:
df = df.drop(['userId', 'name', 'avatar', 'addresses', 'twitterId', 'discord', 'twitter', 'discordId', 'connectedWallet', 'addr'], axis=1)

Next, we identify accounts for the airdrop and tally the XP total.

In [8]:
df = df[df['xp'] > 2000]
round_svy_total = 20000
total_xp = df['xp'].sum()
total_xp

2118635

Distribute this round's 20k SVY across eligible accounts and double-check that it still sums to 20k SVY.

In [9]:
df['proportion'] = df['xp'] / total_xp
df['amount'] = df['proportion'] * round_svy_total
df['amount'].sum()

20000.000000000004

Write the results of round1.

In [10]:
df.to_csv('data/round1.csv')

Done